In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import cholesky
import scipy.special as sci_spe
from time import time

3D simulation of water vapor density with correlation

In [ ]:
def kolmogorov_correlation(r, r0=300):
    '''
    Correlation function of water vapor as a function of distance r (Kolmogorov model).
    Due to the modified Bessel function, this function is very costly to compute.
    '''
    return np.where(r==0, 1, 2**(2/3)/sci_spe.gamma(1/3)*(r/r0)**(1/3)*sci_spe.kv(1/3, r/r0))

In [ ]:
r = np.arange(0, 4, 0.01)
corr = kolmogorov_correlation(r, r0=1)
fit_parameters = np.polyfit(r, corr, 12)
fit_list = np.polyval(fit_parameters, r)
plt.plot(r, corr, label='Kolmogorov')
plt.plot(r, fit_list, label='fit')
plt.legend()
plt.show()

In [ ]:
def correlation_function(r, r0=300):
    '''
    Polynomial approximation of Kolmogorov's correlation. Computation is much faster.
    '''
    return np.where(r>4*r0, 0, np.polyval(fit_parameters, r/r0))

In [ ]:
r0 = 300 #m

N=20
# Generate grid points in 3D space 
x = np.linspace(1, r0, N)
y = np.linspace(1, r0, N)
z = np.linspace(1, r0, N)

# Create meshgrid
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Reshape the grid into a list of 3D coordinates
grid_points = np.column_stack([X.flatten(), Y.flatten(), Z.flatten()])

step0=time()
# Compute pairwise distances between grid points
distances = np.linalg.norm(grid_points[:, None] - grid_points, axis=-1)
print(np.shape(distances))
step1=time()
print(f'distances {step1-step0}')

# Evaluate the 3D correlation function at these distances
correlation_values = correlation_function(distances, r0)
correlation_values=np.nan_to_num(correlation_values, nan=1.0)
step2=time()
print(f'correlation {step2-step1}')

# Create the covariance matrix using the correlation values
covariance_matrix = np.reshape(correlation_values, (len(x)*len(y)*len(z), len(x)*len(y)*len(z)))

# Perform Cholesky decomposition
L = cholesky(covariance_matrix, lower=True)
step3=time()
print(f'cholesky {step3-step2}')

# Generate uncorrelated noise samples
uncorrelated_samples = np.random.randn(len(x)*len(y)*len(z))
step4=time()
print(f'random {step4-step3}')

# Transform uncorrelated samples to correlated samples using Cholesky decomposition
correlated_samples = np.dot(L, uncorrelated_samples)

# Reshape correlated samples back to 3D
correlated_samples_3d = np.reshape(correlated_samples, (len(x), len(y), len(z)))

rho=correlated_samples_3d

In [ ]:
# 3d plot of the fluctuations
fig = plt.figure(figsize = (10,8))
ax = fig.add_subplot(111, projection='3d')
X, Y, Z = np.meshgrid(x, y, z)

kw = {'vmin': rho.min(),
    'vmax': rho.max(),
    'levels': np.linspace(rho.min(), rho.max(), 30)}

# Plot contour surfaces
ax.contourf(X[:, :, -1], Y[:, :, -1], rho[:, :, -1], zdir='z', offset=Z.max(), cmap='jet', **kw)
ax.contourf(X[0, :, :], rho[0, :, :], Z[0, :, :], zdir='y', offset=Y.min(), cmap='jet', **kw)
C = ax.contourf(rho[:, -1, :], Y[:, -1, :], Z[:, -1, :], zdir='x', offset=X.max(), cmap='jet', **kw)

# Set limits of the plot from coord limits
xmin, xmax = X.min(), X.max()
ymin, ymax = Y.min(), Y.max()
zmin, zmax = Z.min(), Z.max()
ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax], zlim=[zmin, zmax])

# Plot edges
edges_kw = dict(color='0.4', linewidth=1, zorder=1e3)
ax.plot([xmax, xmax], [ymin, ymax], zmax, **edges_kw)
ax.plot([xmin, xmax], [ymin, ymin], zmax, **edges_kw)
ax.plot([xmax, xmax], [ymin, ymin], [zmin, zmax], **edges_kw)

# Set labels
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('3D Water Vapor Distribution')
cbar = fig.colorbar(C, ax=ax, location='left')
#cbar = fig.colorbar(scatter, ax=ax, location='left')
cbar.set_label('Water Vapor density (g/m3)')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,8))
ax = fig.add_subplot(111, projection='3d')
X, Y, Z = np.meshgrid(x, y, z)

scatter = ax.scatter(X, Y, Z, c=correlated_samples_3d, cmap='jet')

# Set labels
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('3D Water Vapor Distribution')
cbar = fig.colorbar(scatter, ax=ax, format = '%.5f', location='left')
#cbar.set_label('Water Vapor density (g/m3)')
plt.show()

# Useful functions

In [ ]:
def water_vapor_density(z, rho_0=1, h0=1000):
    '''
    Water vapor density as a function of geopotential height h
    rho_0: Reference mean density of water vapor in g/m³
    h0: The half height for water vapor in m
    '''
    return rho_0 * np.exp(-np.log(2) * (z - 5190) / h0)

In [ ]:
def P_2d(k, r0=300):
    '''
    Proportionnal to the probability distribution of water vapor in 2 dimensions
    k: wavenumber
    '''
    return (1/r0**2+k**2)**(-5/3)

In [ ]:
def P_3d(k, r0=300):
    '''
    Proportionnal to the probability distribution of water vapor in 3 dimensions
    k: wavenumber
    '''
    return (1/r0**2+k**2)**(-11/3)

In [ ]:
def correlation_function_3d(r, r0=300):
    '''
    Fonction de correlation de la vapor d'eau en fonction de la distance r
    '''
    return 2**(2/3)/sci_spe.gamma(1/3)*(r/r0)**(1/3)*sci_spe.kv(1/3, r/r0)

# Mean water vapor density as a function of height

In [ ]:
z = np.linspace(5190, 10000)
rho_0 = 1
h0 = 1000 
plt.plot(water_vapor_density(z, rho_0, h0), z, label=r'$\rho_0$ =' + f'{rho_0} g/m³ , ' + r'$h_0$ = ' + f'{h0} m')
plt.xlabel('Water vapor density (g/m³)')
plt.ylabel('Geopopential height (m)')
plt.legend()

# 2D simulation of water vapor density with correlation using the Cholesky decomposition of the correlation matrix

In [ ]:
from time import time

start = time()
N=70
# Generate grid points in 3D space 
x = np.linspace(1, 600, N)
y = np.linspace(1, 600, N)

# Create meshgrid
X, Y = np.meshgrid(x, y, indexing='ij')

# Reshape the grid into a list of 3D coordinates
grid_points = np.column_stack([X.flatten(), Y.flatten()])
print('initial')
step1 = time()
print(step1-start)

# Compute distances to the origin
distances = np.linalg.norm(grid_points[1:]-[1,1], axis=-1)
print('distances')
step2 = time()
print(step2-step1)

# Evaluate the 3D correlation function in an effective way by using the symmetries in distances between points in a 2D grid
corr = correlation_function_3d(distances)
corr=np.concatenate(([1.0], corr))

l=[corr]
for i in range(1,N):
    for j in range(0,N):
        l.append(corr[j*N:j*N+i+1][::-1])
        l.append(corr[j*N+1:(j+1)*N-i])
corr = np.concatenate(l)

l=[corr]
corr = np.reshape(corr, (N**2, N))
for k in range(1,N):
    for i in range(0,N):
        l.append(corr[i*N:i*N+k+1][::-1].flatten())
        l.append(corr[i*N+1:(i+1)*N-k].flatten())
correlation_values = np.concatenate(l)



# Create the covariance matrix using the correlation values
covariance_matrix = np.reshape(correlation_values, (len(x)*len(y), len(x)*len(y)))
print('correlation_matrix')
step3 = time()
print(step3-step2)

# Perform Cholesky decomposition
L = cholesky(covariance_matrix, lower=True)

print('cholesky')
step4 = time()
print(step4-step3)

# Generate uncorrelated noise samples
uncorrelated_samples = np.random.randn(len(x)*len(y))
print(uncorrelated_samples)

# Transform uncorrelated samples to correlated samples using Cholesky decomposition
correlated_samples = np.dot(L, uncorrelated_samples)

# Reshape correlated samples back to 3D
correlated_samples_2d = np.reshape(correlated_samples, (len(x), len(y)))

h = 4869 # Geopotential height for QUBIC + 1km (m)
rho_correlated = water_vapor_density(h) * 1 + (correlated_samples_2d - np.mean(correlated_samples_2d))

print('correlated_samples')
step5 = time()
print(step5-step4)

In [ ]:
fig=plt.figure()
plt.imshow(rho_correlated, cmap='jet', extent=(1,x.max(),y.max(),1))
plt.colorbar(label='Water Vapor density (g/m3)')
plt.title('Water vapor distribution 1 km over Qubic')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.show()

In [ ]:
rho2 = []
for r in range(1, N):
    rho_r = []
    for j in range(0, N):
        rho_r.append(np.mean(rho_correlated[j][0:N-r]*rho_correlated[j][r:N]))
    rho2.append(np.mean(rho_r))
rho2 = np.array(rho2)

rho_corr_transposed = np.transpose(rho_correlated)
rho2_t = []
for r in range(1, N):
    rho_r = []
    for j in range(0, N):
        rho_r.append(np.mean(rho_correlated[j][0:N-r]*rho_correlated[j][r:N]))
    rho2_t.append(np.mean(rho_r))
rho2_t = np.array(rho2_t)

D = ((rho2+rho2_t)/2 - np.mean(rho_correlated)**2)

In [ ]:
fig=plt.figure()
plt.plot(x[:-1], D)
plt.plot(x[:-1], correlation_function(x[:-1]))
plt.show()

In [ ]:
# Rotation matrix
def rotation_matrix(r_i):
    rx, ry, rz = r_i
    sqrt = np.sqrt((ry**2 + rz**2)**2 + (rx*ry)**2 + (rx*rz)**2)
    return np.array([[(ry**2+rz**2)/sqrt, 0, rx], [-rx*ry/sqrt, rz/np.sqrt(ry**2+rz**2), ry], [-rx*rz/sqrt, -ry/np.sqrt(ry**2+rz**2), rz]])

In [ ]:
# Functions to compute boundaries of integration
def boundaries_xi(yi, zi, r_i, freq):
    sqrt = np.sqrt(gaussian_beam_width(zi, freq)**2 - (yi - zi*r_i[1]/r_i[2])**2)
    offset = zi*r_i[0]/r_i[2]
    return (-sqrt+offset, sqrt+offset)

def boundaries_yi(zi, r_i, freq):
    wz = gaussian_beam_width(zi, freq)
    offset = zi*r_i[1]/r_i[2]
    return (-wz+offset, wz+offset)

In [ ]:
def bounds(z_min, z_max, r_i, freq):
    wz = gaussian_beam_width(z_max, freq)
    l_bounds = []
    u_bounds = []
    l_bounds.append(-wz-z_max*np.abs(r_i[0]/r_i[2])) # xi
    u_bounds.append(wz+z_max*np.abs(r_i[0]/r_i[2]))
    l_bounds.append(-wz-z_max*np.abs(r_i[1]/r_i[2])) # yi
    u_bounds.append(wz+z_max*np.abs(r_i[1]/r_i[2]))
    l_bounds.append(z_min) # zi
    u_bounds.append(z_max)
    return l_bounds, u_bounds

def is_in_the_beam(arr, r_i, freq):
    xi, yi, zi = arr
    xi_min, xi_max = boundaries_xi(yi, zi, r_i, freq)
    if xi >= xi_min and xi <= xi_max:
        yi_min, yi_max = boundaries_yi(zi, r_i, freq)
        if yi >= yi_min and yi <= yi_max:
            return True
    return False

# Function to generate integration points within variable-dependent bounds
def integration_points(h_min, h_max, r_i, r_j, el, freq, n):
    '''
    Number of points is 2**n
    '''
    
    sampler_i = qmc.Sobol(d=3, scramble=False)
    points_i = sampler_i.random_base2(m=n)
    sampler_j = qmc.Sobol(d=3, scramble=False)
    points_j = sampler_j.random_base2(m=n)
    print(points_j)
    
    z_min = (h_min - h) / np.sin(el)
    z_max = (h_max - h) / np.sin(el)

    l_bounds_i, u_bounds_i = bounds(z_min, z_max, (0,0,1), freq) ##
    scaled_points_i = qmc.scale(points_i, l_bounds_i, u_bounds_i)
    l_bounds_j, u_bounds_j = bounds(z_min, z_max, (0,0,1), freq) ##
    scaled_points_j = qmc.scale(points_j, l_bounds_j, u_bounds_j)

    # Mask the points not in the beams
    masked_points_i = []
    for arr in scaled_points_i:
        if is_in_the_beam(arr.tolist(), (0,0,1), freq): ##
            masked_points_i.append(arr.tolist())
    random.shuffle(masked_points_i)
    masked_points_j = []
    for arr in scaled_points_j:
        #print(arr)
        if is_in_the_beam(arr.tolist(), (0,0,1), freq): ##
            masked_points_j.append(arr.tolist())
    random.shuffle(masked_points_j)

    masked_points_i = np.transpose(rotation_matrix(r_i) @ np.transpose(np.array(masked_points_i))).tolist() ##
    masked_points_j = np.transpose(rotation_matrix(r_j) @ np.transpose(np.array(masked_points_j))).tolist() ##
    
    masked_points = []
    scaled_points = []########
    for k in range(min(len(masked_points_i), len(masked_points_j))):
        masked_points.append(masked_points_i[k] + masked_points_j[k])
    for k in range(2**n):
        scaled_points.append(scaled_points_i[k].tolist() + scaled_points_j[k].tolist())
    
    print(len(masked_points))

    return np.array(masked_points)


In [ ]:
def atm_correlations_gaussian_qmc(h_min, h_max, r_i, r_j, el, freq, n):
    points = integration_points(h_min, h_max, r_i, r_j, el, freq, n)

    int = 0
    for arr in points:
        int += integrand_correlations_gaussian(arr, r_i, r_j, el, freq)

    #volume = (pi * integrale(wz)**2)**2 The second square is because there are two beams
    volume = (np.pi * scipy.integrate.quad(lambda z, freq : gaussian_beam_width(z, freq)**2, h_min-h, h_max-h, args=freq)[0])**2
    
    return np.shape(points)[0], molecular_absorption_coeff**2 * int / np.shape(points)[0] * volume

In [ ]:
def is_in_the_beam(arr, r_i, freq):
    xi, yi, zi = arr
    xi_min, xi_max = boundaries_xi(yi, zi, r_i, freq)
    if xi >= xi_min and xi <= xi_max:
        yi_min, yi_max = boundaries_yi(zi, r_i, freq)
        if yi >= yi_min and yi <= yi_max:
            return True
    return False

In [ ]:
# Rotation matrix in six dimensions (two idependant 3D rotations, one for each beam)
def rotation_matrix_six(r_i, r_j):
    rxi, ryi, rzi = r_i
    rxj, ryj, rzj = r_j
    sqrt_i = np.sqrt((ryi**2 + rzi**2)**2 + (rxi*ryi)**2 + (rxi*rzi)**2)
    sqrt_j = np.sqrt((ryj**2 + rzj**2)**2 + (rxj*ryj)**2 + (rxj*rzj)**2)

    L1 = [(ryi**2+rzi**2)/sqrt_i, -rxi*ryi/sqrt_i, -rxi*rzi/sqrt_i, 0, 0, 0]
    L2 = [0, rzi/np.sqrt(ryi**2+rzi**2), -ryi/np.sqrt(ryi**2+rzi**2), 0, 0, 0]
    L3 = [rxi, ryi, rzi, 0, 0, 0]
    L4 = [0, 0, 0, (ryj**2+rzj**2)/sqrt_j, -rxj*ryj/sqrt_j, -rxj*rzj/sqrt_j]
    L5 = [0, 0, 0, 0, rzj/np.sqrt(ryj**2+rzj**2), -ryj/np.sqrt(ryj**2+rzj**2)]
    L6 = [0, 0, 0, rxj, ryj, rzj]
    Rxy = np.array([L1, L2, L3, L4, L5, L6])
    return Rxy